**Description**

Для 151 406 договоров страхования транспортных средств известны значения ряда признаков, в том числе пол, возраст, стаж вождения и коэффициент бонус-малус водителя, тип, марка, модель, год выпуска, страна – производитель, мощность и объем двигателя, а также признак target, равный 1, если заключение договора с клиентом является рисковым, и 0 в противном случае (файл insclass_train.csv).

Требуется построить модель, предсказывающую значение признака target для 22 624 договоров из тестового набора данных (файл insclass_test.csv).

Метрикой качества является площадь под ROC-кривой (AUC).

Одной из особенностей данной задачи является то, что почти для всех марок присутствуют разные модели, представляющие их.

In [252]:
import pandas as pd
import numpy as np
import seaborn as sns

In [253]:
import matplotlib.pyplot as plt
%matplotlib inline

In [254]:
train = pd.read_csv(f'C:/Users/aleks/Desktop/insclass_train.csv')

In [255]:
test = pd.read_csv(f'C:/Users/aleks/Desktop/insclass_test.csv')

In [256]:
train

,variable_1,variable_2,variable_3,variable_4,variable_5,variable_6,variable_7,variable_8,variable_9,variable_10,...,variable_20,variable_21,variable_22,variable_23,variable_24,variable_25,variable_26,variable_27,variable_28,target
0,w200,0,0,14,q2,98.0,NaN,0.0,NaN,0,...,C,j2,h45,0,0.0,0,1,19.323463,t1,0
1,w160,0,0,7,q11,106.0,NaN,0.0,NaN,0,...,C,j33,h234,0,1.0,0,1,41.177900,t1,0
2,w200,0,0,4,q3,123.0,NaN,0.0,NaN,0,...,B,j12,h28,0,0.0,0,1,3.614395,t1,0
3,w200,0,0,9,q3,102.0,NaN,0.0,NaN,0,...,C,j12,h64,0,1.0,0,0,49.041674,t1,0
4,w200,0,0,18,q20,117.0,NaN,0.0,NaN,0,...,C,j111,h991,0,1.0,0,0,17.909612,t1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151401,w48,0,0,3,q3,123.0,NaN,0.0,NaN,0,...,C,j0,h0,0,1.0,0,1,12.200897,t1,0
151402,w160,0,0,6,q3,123.0,1591.0,NaN,150000.0,0,...,C,j0,h0,0,1.0,0,1,31.787491,t1,0
151403,w200,0,0,7,q2,158.0,1998.0,0.0,NaN,0,...,C,j4,h14,0,0.0,0,1,7.883271,t1,0
151404,w48,0,0,3,q4,82.0,NaN,NaN,NaN,0,...,B,j7,h31,0,1.0,0,1,20.731525,t1,0


**Описание полей**

В обучающем наборе данных для каждого договора известны следующие поля:

variable_1 - агрегированный коэффициент бонус-малус (повышающий или понижающий стоимость полиса в зависимости от аварийности в предыдущие периоды);\
variable_2 - индикатор расторжения договора по инициативе страхователя (клиента);\
variable_3 - индикатор расторжения договора по инициативе страховщика (страховой компании);\
variable_4 - идентификатор года выпуска транспортного средства;\
variable_5 - идентификатор страны - производителя транспортного средства;\
variable_6 - мощность двигателя в лошадиных силах;\
variable_7 - объем двигателя в куб. см;\
variable_8 - идентификатор стороны расположения руля (левый или правый);\
variable_9 - пробег транспортного средства, покрываемый гарантией производителя;\
variable_10 - индикатор действия гарантии на транспортное средство;\
variable_11 - "мультидрайв" - индикатор допуска к управлению транспортным средством более одного водителя;\
variable_12 - возраст транспортного средства (в мес.);\
variable_13 - возраст водителя с максимальным стажем;\
variable_14 - коэффициент возраст-стаж;\
variable_15 - коэффициент краткосрочности;\
variable_16 - коэффициент мощности;\
variable_17 - коэффициент "мультидрайв";\
variable_18 - территориальный коэффициент;\
variable_19 - коэффициент "КНДР";\
variable_20 - идентификатор канала продаж;\
variable_21 - марка транспортного средства;\
variable_22 - модель транспортного средства;\
variable_23 - индикатор отечественных транспортных средств;\
variable_24 - пол водителя с максимальным коэффициентом "возраст-стаж";\
variable_25 - индикатор пролонгации;\
variable_26 - индикатор совпадения собственника транспортного средства и водителя;\
variable_27 - стаж водителя с максимальным коэффициентом "возраст-стаж";\
variable_28 - тип транспортного средства;\
target - класс риска, равный 1, если заключение договора с клиентом является рисковым, и 0 в противном случае.\
В тестовом наборе данных каждому договору соответствует уникальный идентификатор id.

In [257]:
from sklearn.preprocessing import LabelEncoder

In [258]:
le = LabelEncoder()
variable_5 = le.fit_transform(train['variable_5'])

In [259]:
variable_5

array([12,  3, 23, ..., 12, 31,  1])

In [260]:
train["variable_5"] = variable_5

In [261]:
variable_1 = le.fit_transform(train['variable_1'])
variable_20 = le.fit_transform(train['variable_20'])
variable_21 = le.fit_transform(train['variable_21'])
variable_22 = le.fit_transform(train['variable_22'])
variable_28 = le.fit_transform(train['variable_28'])

In [262]:
train["variable_1"] = variable_1
train["variable_20"] = variable_20
train["variable_21"] = variable_21
train["variable_22"] = variable_22
train["variable_28"] = variable_28

In [263]:
train.isnull().describe()

,variable_1,variable_2,variable_3,variable_4,variable_5,variable_6,variable_7,variable_8,variable_9,variable_10,...,variable_20,variable_21,variable_22,variable_23,variable_24,variable_25,variable_26,variable_27,variable_28,target
count,151406,151406,151406,151406,151406,151406,151406,151406,151406,151406,...,151406,151406,151406,151406,151406,151406,151406,151406,151406,151406
unique,1,1,1,1,1,2,2,2,2,1,...,1,1,1,1,2,1,1,2,1,1
top,False,False,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
freq,151406,151406,151406,151406,151406,151295,95639,121507,134436,151406,...,151406,151406,151406,151406,149199,151406,151406,149339,151406,151406


In [264]:
for i in train:
    train[i].fillna(train[i].median(), inplace=True)

In [265]:
train

,variable_1,variable_2,variable_3,variable_4,variable_5,variable_6,variable_7,variable_8,variable_9,variable_10,...,variable_20,variable_21,variable_22,variable_23,variable_24,variable_25,variable_26,variable_27,variable_28,target
0,8,0,0,14,12,98.0,1598.0,0.0,100000.0,0,...,2,92,910,0,0.0,0,1,19.323463,0,0
1,4,0,0,7,3,106.0,1598.0,0.0,100000.0,0,...,2,123,679,0,1.0,0,1,41.177900,0,0
2,8,0,0,4,23,123.0,1598.0,0.0,100000.0,0,...,1,23,728,0,0.0,0,1,3.614395,0,0
3,8,0,0,9,23,102.0,1598.0,0.0,100000.0,0,...,2,23,1112,0,1.0,0,0,49.041674,0,0
4,8,0,0,18,13,117.0,1598.0,0.0,100000.0,0,...,2,14,1467,0,1.0,0,0,17.909612,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151401,12,0,0,3,23,123.0,1598.0,0.0,100000.0,0,...,2,0,0,0,1.0,0,1,12.200897,0,0
151402,4,0,0,6,23,123.0,1591.0,0.0,150000.0,0,...,2,0,0,0,1.0,0,1,31.787491,0,0
151403,8,0,0,7,12,158.0,1998.0,0.0,100000.0,0,...,2,130,390,0,0.0,0,1,7.883271,0,0
151404,12,0,0,3,31,82.0,1598.0,0.0,100000.0,0,...,1,163,760,0,1.0,0,1,20.731525,0,0


In [269]:
train.isnull().describe()

,variable_1,variable_2,variable_3,variable_4,variable_5,variable_6,variable_7,variable_8,variable_9,variable_10,...,variable_20,variable_21,variable_22,variable_23,variable_24,variable_25,variable_26,variable_27,variable_28,target
count,151406,151406,151406,151406,151406,151406,151406,151406,151406,151406,...,151406,151406,151406,151406,151406,151406,151406,151406,151406,151406
unique,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,151406,151406,151406,151406,151406,151406,151406,151406,151406,151406,...,151406,151406,151406,151406,151406,151406,151406,151406,151406,151406


In [270]:
y = train["target"]
X = train.iloc[:, 0:28]

Узнаем, какие именно из признаков неинформативные. Это можно попробовать определить, посчитав для каждого признака ROC AUC:

In [271]:
from sklearn.metrics import roc_auc_score
non_informative = []

for i in range(X.shape[1]):
    auc = roc_auc_score(y, X.iloc[:, i])

    if abs(auc - 0.5) < 0.02:
        print(f"variable_{i+1} : {auc = :.3f} -> non_informative")
        non_informative.append(f"variable_{i}")

    else:
        print(f"variable_{i+1} : {auc = :.3f}")

variable_1 : auc = 0.548
variable_2 : auc = 0.497 -> non_informative
variable_3 : auc = 0.500 -> non_informative
variable_4 : auc = 0.491 -> non_informative
variable_5 : auc = 0.491 -> non_informative
variable_6 : auc = 0.506 -> non_informative
variable_7 : auc = 0.488 -> non_informative
variable_8 : auc = 0.500 -> non_informative
variable_9 : auc = 0.496 -> non_informative
variable_10 : auc = 0.502 -> non_informative
variable_11 : auc = 0.559
variable_12 : auc = 0.488 -> non_informative
variable_13 : auc = 0.415
variable_14 : auc = 0.522
variable_15 : auc = 0.500 -> non_informative
variable_16 : auc = 0.506 -> non_informative
variable_17 : auc = 0.560
variable_18 : auc = 0.505 -> non_informative
variable_19 : auc = 0.452
variable_20 : auc = 0.484 -> non_informative
variable_21 : auc = 0.498 -> non_informative
variable_22 : auc = 0.511 -> non_informative
variable_23 : auc = 0.501 -> non_informative
variable_24 : auc = 0.505 -> non_informative
variable_25 : auc = 0.508 -> non_informativ

In [272]:
for i in non_informative:
    train.drop(i, axis=1, inplace=True)

In [273]:
variable_5 = le.fit_transform(test['variable_5'])
variable_1 = le.fit_transform(test['variable_1'])
variable_20 = le.fit_transform(test['variable_20'])
variable_21 = le.fit_transform(test['variable_21'])
variable_22 = le.fit_transform(test['variable_22'])
variable_28 = le.fit_transform(test['variable_28'])

test["variable_5"] = variable_5
test["variable_1"] = variable_1
test["variable_20"] = variable_20
test["variable_21"] = variable_21
test["variable_22"] = variable_22
test["variable_28"] = variable_28

for i in test:
    test[i].fillna(test[i].median(), inplace=True)

In [276]:
test.isnull().describe()

,id,variable_1,variable_2,variable_3,variable_4,variable_5,variable_6,variable_7,variable_8,variable_9,...,variable_19,variable_20,variable_21,variable_22,variable_23,variable_24,variable_25,variable_26,variable_27,variable_28
count,22624,22624,22624,22624,22624,22624,22624,22624,22624,22624,...,22624,22624,22624,22624,22624,22624,22624,22624,22624,22624
unique,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,22624,22624,22624,22624,22624,22624,22624,22624,22624,22624,...,22624,22624,22624,22624,22624,22624,22624,22624,22624,22624


In [277]:
for i in non_informative:
    test.drop(i, axis=1, inplace=True)

In [278]:
y = train["target"]
X = train.iloc[:, 0:8]

Проведем балансировку классов

In [279]:
class_counts = train['target'].value_counts()

majority_class = class_counts.idxmax()
minority_class = class_counts.idxmin()

df_majority = train[train['target'] == majority_class]

df_minority = train[train['target'] == minority_class]

df_minority_duplicates = pd.concat([df_minority] * (class_counts[majority_class] // class_counts[minority_class]), ignore_index=True)

df_balanced = pd.concat([df_majority, df_minority_duplicates], ignore_index=True)

In [280]:
y_balanced = df_balanced["target"]
df_balanced.drop('target', axis=1, inplace=True)
X_balanced = df_balanced

Чтобы оценка качества моделей была честной, разобьем наши данные на обучение и тест. Метрики будем считать на выборке, которую модель не видела на этапе обучения.

In [281]:
from sklearn.model_selection import train_test_split

In [282]:
X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=35)

Для перебора гиперпараметров выделим отдельную **валидационную выборку** из обучающей:
 - на X_learn, y_learn будем обучать модели
 - по качеству на X_val, y_val будем выбирать лучшую

In [283]:
X_learn, X_val, y_learn, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=35)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [95]:
from itertools import product

# возможные значения гиперпараметров
param_values = {
    "max_depth" : [None, 3, 4, 5, 10, 15],
    "criterion" : ["gini", "entropy", "log_loss"],
    "min_samples_leaf" : [1, 30, 50, 100],
    "min_samples_split" : [2, 50, 100, 200],
    "max_features" : ["sqrt", "log2", None]
}
best_score = 0.5
best_model = None

iterator = product(
    param_values["max_depth"],
    param_values["criterion"],
    param_values["min_samples_leaf"],
    param_values["min_samples_split"],
    param_values["max_features"]
)

for max_depth, criterion, min_samples_leaf, min_samples_split, max_features in iterator:
    # словарь с гиперпараметрами
    params = dict(
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        min_samples_split=min_samples_split
        )

    # обучение модели на X_learn, y_learn
    tree = RandomForestClassifier(**params, random_state=35)
    tree.fit(X_learn, y_learn)

    # оценка качества на X_val, y_val
    proba = tree.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, proba)

    if auc > best_score:
        best_score = auc
        best_model = tree

params = best_model.get_params()
print((
    f"Лучшая модель:\n"
    f"max_depth = {params['max_depth']}\n"
    f"criterion = {params['criterion']}\n"
    f"min_samples_leaf = {params['min_samples_leaf']}\n"
    f"min_samples_split = {params['min_samples_split']}\n"
    f"max_features = {params['max_features']}"
    ))

Лучшая модель:
max_depth = 15
criterion = gini
min_samples_leaf = 1
min_samples_split = 200
max_features = auto


In [284]:
pred_train = best_model.predict_proba(X_train)[:, 1]
pred_test = best_model.predict_proba(X_test)[:, 1]

print((
    f"ROC AUC на обучающей выборке: {roc_auc_score(y_train, pred_train):.3f}\n"
    f"ROC AUC на тестовой выборке:  {roc_auc_score(y_test, pred_test):.3f}"
))

ROC AUC на обучающей выборке: 0.740
ROC AUC на тестовой выборке:  0.737


In [285]:
decision_tree = RandomForestClassifier(max_depth = 15, criterion = 'gini', min_samples_leaf = 1,min_samples_split = 200)
decision_tree.fit(X_balanced, y_balanced)
y_pred = decision_tree.predict(test.iloc[:, 1:])

In [ ]:
id = test["id"]

In [ ]:
test.drop('id', axis=1, inplace=True)

In [ ]:
sol = pd.DataFrame({'id': id,
                    'target': y_pred})
sol

In [212]:
sol.to_csv('end.csv', index=False)